# Professional Services Retreat | GenAI Workshop

# Queries

Example query selection will make or break your Text2Cypher application. The examples must cover the scope of anticipated questions, while not being too numerous that they eat up the context window. They also must represent the significant nodes, relationships, properties and traversals related to the use cases.

Here we will take a look at the queries we curated for the IQS Text2Cypher agent. These queries may be found in the [queries.yml](../../data/iqs/queries/queries.yml) file located at `data/iqs/queries/queries.yml`. 

You may also view the patient journey [queries.yml](../../data/patient_journey/queries/queries.yml) file at `data/patient_journey/queries/queries.yml`. 

## Imports

In [1]:
import sys

sys.path.append("../../")
from src.ps_genai_agents.prompts.queries import get_example_queries

## First Example

In [2]:
iqs_queries = get_example_queries("../../data/iqs/queries/queries.yml")

In [6]:
def pretty_print(query) -> None:
    # we need double curley braces for injecting into a formatted string later. 
    # We can remove for this notebook example.
    cql = query.get("assistant").replace("{{", "{")
    cql = cql.replace("}}", "}")
    print(query.get("human"), "\n")
    print(cql)

We decided to include this query in the examples because it demonstrates a few concepts
* **aggregation**: We are asking for a proportion to be calculated
* **multiple filters**: We must filter on gender, complaint topic and vehicle type

In [7]:
pretty_print(iqs_queries[0])

What is the proportion of male to female complainants about cup holders in the Honda Odyssey? 

MATCH (v:Verbatim {make: "Honda", model: "Odyssey"})
  WHERE v.verbatimText CONTAINS "cup holder"
WITH SUM(COUNT {MATCH (v:Verbatim) WHERE v.gender = "Male" RETURN v}) AS males, 
     SUM(COUNT {MATCH (v:Verbatim) WHERE v.gender = "Female" RETURN v}) AS females
RETURN males, females, toFloat(males) / (CASE WHEN females = 0 THEN 1 ELSE females END)  AS maleToFemale


## Second Example

We chose this query because it demostrates 
* **multiple filters**: We must filter on many properties
* **null checks**: We only collect verbatims that are not empty
* **UNION**: We must use UNION to gather male and female results
* **COLLECT**: We must collect the verbatims into a list for future processing

In [8]:
pretty_print(iqs_queries[-2])

How similar or different are the verbatims for males and females aged 30-34 for all Acura models? 

MATCH (v:Verbatim) 
WHERE v.gender = "Male" AND v.make = "Acura" AND v.minAge >= 30 AND v.maxAge <= 34 AND isEmpty(v.verbatim) = false
WITH collect(v.verbatim) as content
RETURN "Male" as gender, content LIMIT 50
UNION
MATCH (v:Verbatim) 
WHERE v.gender = "Female" AND v.make = "Acura" AND v.minAge >= 30 AND v.maxAge <= 34 AND isEmpty(v.verbatim) = false
WITH collect(v.verbatim) as content
RETURN "Female" as gender, content LIMIT 50


## Graph Exploration

Explore the [queries.yml](../../data/iqs/queries/queries.yml) file and run these queries for yourself to get a feel for the data. 

Notice that we don't include simple queries for question such as 
* "How many vehicle types are there?"

or
* "How many nodes are in the graph?"

The Cypher for these questions is simple and should be inferrable by more complicated examples provided to the LLM. 

You may test these simple queries later on and see that the Text2Cypher agent is capable of answering them just fine.